# Double pendulum

In [ ]:
using DifferentialEquations
using Plots

### Equations of motion

$$
\mathcal{H} = \dfrac{1}{2}
\dfrac{p_1^2 + 2 p_2^2 - 2 p_1 p_2 \cos(\theta_1 - \theta_2)}{2 - \cos(\theta_1 - \theta_2)^2} +
3 - 2 \cos{\theta_1} - \cos{\theta_2}
$$


In [ ]:
function dpendulum(du, u, p, t)
    cdiff = cos(u[1] - u[2])
    detM = 2 - cdiff^2
    W = (sin(u[1] - u[2]) / detM^2) * ((u[3]^2 + 2*u[4]^2)*cdiff - u[3]*u[4]*(4 - detM))
    du[1] = (u[3] - u[4]*cdiff) / detM;
    du[2] = (2*u[4] - u[3]*cdiff) / detM
    du[3] = -2*sin(u[1]) + W
    du[4] = -sin(u[2]) - W
end

### Constant of motion

In [ ]:
function energy(u)
    e = (1/2)*(u[3]^2 + 2*u[4]^2 - 2*u[3]*u[4]*cos(u[1] - u[2]))/(2 - cos(u[1] - u[2])^2) + 3 - 2*cos(u[1]) - cos(u[2])
end

### Initial conditions

In [ ]:
u0 = [0.0, pi/2, 0.0, 0.0]
#u0 = [pi/3, 0*pi/2, 0.0, 0.0]

energy(u0)

### Time scale

In [ ]:
t_max = 384.0
t_step = 0.02 

### Numerical solution

In [ ]:
prob = ODEProblem(dpendulum, u0, (0.0, t_max))
sol = solve(prob, Vern7(), adaptive=false, dt=t_step)
data = reduce(hcat, sol.u)'

### Phase section $(\theta_1, p_1)$

In [ ]:
plotOrbit = scatter(
    data[:,1], data[:,3],
    aspect_ratio = 1,
    xlabel = "\\theta_1",
    ylabel = "\\pi_1",
    plot_title = "Double pendulum",
    markersize = 0.4,
    alpha = 0.15,
    legend = false,
    dpi = 300
)
annotate!(
    xlims(plotOrbit)[1] + 0.12*(xlims(plotOrbit)[2] - xlims(plotOrbit)[1]),
    ylims(plotOrbit)[2] - 0.03*(ylims(plotOrbit)[2] - ylims(plotOrbit)[1]),
    text("E = $(round(energy(u0), digits=4))", :red, :right, 4)
)

### Integration error

In [ ]:
plotEnergy = scatter(
    sol.t,
    map(eachrow(data)) do s
        energy(s) - energy(u0)
    end,
    xlabel = "t",
    ylabel = "Energy error - \\Delta E",
    plot_title = "Double pendulum",
    markersize = 0.5,
    alpha = 0.25,
    legend = false,
    dpi = 300
)